In [10]:
import re

import pandas as pd
from sklearn.cluster import HDBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle

In [2]:
vpr_df = pd.read_csv('../output/vacancy_professional_roles.csv')
vks_df = pd.read_csv('../output/vacancy_key_skills.csv')

In [3]:
vpr_df

,vacancy_id,professional_role_name
0,108866195,"Программист, разработчик"
1,109005977,"Программист, разработчик"
2,107709799,"Программист, разработчик"
3,108983317,"Программист, разработчик"
4,108127100,"Программист, разработчик"
...,...,...
8484,109683551,Системный администратор
8485,106712051,Системный администратор
8486,108058748,Системный администратор
8487,107789115,Системный администратор


In [4]:
vks_df

,vacancy_id,key_skill_name
0,109005977,PostgreSQL
1,109005977,C#
2,109005977,.NET 6
3,109005977,EF Core
4,109005977,MS SQL
...,...,...
39207,109687310,Администрирование серверов Windows
39208,109687310,VMware
39209,109687310,Информационная безопасность
39210,109687310,Windows Os


In [3]:
df = pd.merge(vpr_df, vks_df, on='vacancy_id')
df

,vacancy_id,professional_role_name,key_skill_name
0,109005977,"Программист, разработчик",PostgreSQL
1,109005977,"Программист, разработчик",C#
2,109005977,"Программист, разработчик",.NET 6
3,109005977,"Программист, разработчик",EF Core
4,109005977,"Программист, разработчик",MS SQL
...,...,...,...
39222,109687310,Системный администратор,Администрирование серверов Windows
39223,109687310,Системный администратор,VMware
39224,109687310,Системный администратор,Информационная безопасность
39225,109687310,Системный администратор,Windows Os


In [4]:
df = shuffle(df, random_state=42)
df

,vacancy_id,professional_role_name,key_skill_name
13295,107315897,Системный инженер,Redis
37544,110614819,Системный администратор,Ремонт ПК
962,108816618,"Программист, разработчик",Reindexer
37455,110913528,Системный администратор,RTPS
17844,109463246,"Программист, разработчик",HTML
...,...,...,...
6265,103911284,Тестировщик,Анализ данных
11284,108519437,DevOps-инженер,DevOps
38158,110309046,Системный администратор,Настройка сетевых подключений
860,107656399,"Программист, разработчик",C#


In [5]:
def process_vacancy_data(df):
    """
    Обрабатывает DataFrame с колонками vacancy_id, professional_role_name, key_skill_name.
    Возвращает DataFrame, где `professional_role_name` и `key_skill_name` сгруппированы по vacancy_id и обработаны.
    """

    # === Шаг 1: Группировка по vacancy_id ===
    df = df.groupby('vacancy_id').agg({
        'professional_role_name': lambda x: list(set(x)),  # Убираем дубликаты
        'key_skill_name': lambda x: list(set(x))           # Убираем дубликаты
    }).reset_index()

    # === Шаг 2: Очистка текста в key_skill_name ===
    def clean_symbols(text):
        """
        Очищает текст от лишних символов, нормализует пробелы и приводит к нижнему регистру.
        """
        text = re.sub(r'[^\w\s]', ' ', text)  # Удаляем все символы, кроме букв, цифр и пробелов
        text = re.sub(r'\s+', ' ', text).strip()  # Нормализуем пробелы
        return text.lower()  # Приводим к нижнему регистру

    df['key_skill_name'] = df['key_skill_name'].apply(lambda skills: [clean_symbols(skill) for skill in skills])

    # === Шаг 3: Разделение перечисленных навыков ===
    def split_skills(skill_list):
        """
        Разделяет перечисленные навыки в списке на отдельные элементы.
        """
        result = []
        for skill in skill_list:
            result.extend(re.split(r'[,\;/]', skill))  # Разделяем по , ; /
        return [skill.strip() for skill in result if skill.strip()]  # Убираем пустые строки

    df['key_skill_name'] = df['key_skill_name'].apply(split_skills)

    # === Шаг 4: Устранение опечаток через кластеризацию ===
    def cluster_skills(skills):
        """
        Группирует похожие навыки и выбирает представителя для каждого кластера.
        """
        # Преобразование навыков в TF-IDF матрицу
        vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))  # биграммы и триграммы
        X = vectorizer.fit_transform(skills)

        # Кластеризация HDBSCAN
        clusterer = HDBSCAN(min_cluster_size=2, metric='euclidean')
        labels = clusterer.fit_predict(X)

        # Формирование DataFrame для кластеров
        clustered = pd.DataFrame({'original_skill': skills, 'cluster': labels})

        # Выбор представителя кластера
        representatives = clustered.groupby('cluster')['original_skill'].agg(
            lambda group: group.value_counts().idxmax()
        )

        # Присвоение представителя навыку
        clustered['representative_skill'] = clustered['cluster'].map(representatives)

        # Для шума оставляем оригинальные значения
        clustered['representative_skill'] = clustered.apply(
            lambda row: row['original_skill'] if row['cluster'] == -1 else row['representative_skill'], axis=1
        )

        # Словарь для замены
        return clustered.set_index('original_skill')['representative_skill'].to_dict()

    # Собираем все навыки в один список для кластеризации
    all_skills = [skill for sublist in df['key_skill_name'] for skill in sublist]
    skill_map = cluster_skills(all_skills)

    # Замена навыков в каждой вакансии
    df['key_skill_name'] = df['key_skill_name'].apply(lambda skills: [skill_map[skill] for skill in skills])

    return df

In [6]:
df = process_vacancy_data(df)
df

,vacancy_id,professional_role_name,key_skill_name
0,45163827,[Системный администратор],"[nginx, cистемы управления базами данных, mong..."
1,46823861,"[Программист, разработчик]","[flask, docker, postgresql, redis, python, linux]"
2,46954274,[Аналитик],"[bpmn, rest, системный анализ, работа с требов..."
3,54961293,[Системный инженер],"[ansible, nginx, postgresql, ci cd, elk, bash,..."
4,66766808,"[Программист, разработчик]","[node js, rest api, nestjs, javascript, typesc..."
...,...,...,...
5414,111082906,[Системный администратор],[желательно знания сетевой инфраструктуры опыт...
5415,111099873,[Системный администратор],"[настройка сетевых подключений, антивирусная з..."
5416,111112257,[Системный администратор],"[системный администратор, ansible, node js, ng..."
5417,111124064,[DevOps-инженер],"[ssl, docker, настройка dns, ci cd, grafana, d..."


In [7]:
# X содержит vacancy_id и professional_role_name
X = df[['professional_role_name']]
# y содержит key_skill_name
y = df['key_skill_name']
print(X.shape)
print(y.shape)


(5419, 1)
(5419,)


In [8]:
# Инициализируем бинаризаторы
role_binarizer = MultiLabelBinarizer()
skill_binarizer = MultiLabelBinarizer()

# Преобразуем professional_role_name и key_skill_name в бинарные представления
X_roles = role_binarizer.fit_transform(X['professional_role_name'])
y_skills = skill_binarizer.fit_transform(y)

# Обновляем X (удаляем списки, добавляем бинарные представления)
X = pd.DataFrame(X_roles, columns=role_binarizer.classes_)

# Выводим размеры
print("Размеры X:", X.shape)
print("Размеры y:", y_skills.shape)

Размеры X: (5419, 100)
Размеры y: (5419, 3505)


In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_skills, test_size=0.2, random_state=42)

# Инициализация и обучение модели
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Предсказание и оценка
y_pred = clf.predict(X_test)
score = f1_score(y_test, y_pred, average='micro')
print("F1-Score:", score)

In [7]:
def clean_symbols(text):
    """
    Очищает текст от лишних символов, нормализует пробелы и приводит к нижнему регистру.
    """
    text = re.sub(r'[^\w\s]', ' ', text)  # Удаляем все символы, кроме букв, цифр и пробелов
    text = re.sub(r'\s+', ' ', text).strip()  # Нормализуем пробелы
    return text.lower()  # Приводим к нижнему регистру

In [8]:
skills = [
    "HTML, CSS, JS!",   # навык с лишними символами
    "  Git    is  cool  ",   # лишние пробелы
    "Git"                # разный регистр
]
skills = df['key_skill_name']

cleaned_skills = [clean_symbols(skill) for skill in skills]
print(cleaned_skills)

['redis', 'ремонт пк', 'reindexer', 'rtps', 'html', 'linux', 'разработка по', 'bash', 'linux', 'css', 'гост 34', 'javascript', 'системный анализ', 'ci cd', 'html', 'gsm', 'ef core', 'user story', 'настройка пк', 'solid', 'разработка по', 'навыки презентации', 'интерпретация данных', 'devops', 'gitlab ci', 'ci cd', 'linux', 'javascript', 'системный подход', 'оптимизация бизнес процессов', 'mysql', 'ansible', 'influxdb', 'сопровождение строительства', 'test case', 'настройка dns', 'npi', 'функциональное тестирование', 'unix', 'docker', 'kafka', 'руководство коллективом', 'управленческая отчетность', 'кортеос', 'nginx', 'javascript', 'обучение и развитие', 'системный анализ', 'elasticsearch', 'docker', 'техническая поддержка', 'администрирование postgresql', 'ведение деловой переписки', 'c', 'mlag', 'docker', 'zabbix', 'agile', 'развитие продаж', 'прямые продажи', 'строительство', 'typescript', 'project management', 'rest api', 'asyncio', 'разработка функциональных моделей', 'centos', 'um

In [9]:
def split_skills(skill_string):
    """
    Разделяет строку с перечислением навыков на отдельные навыки.
    """
    # Используем регулярное выражение для разделения по , ; или /
    skills = re.split(r'[,\;]', skill_string)
    # Убираем лишние пробелы у каждого навыка
    return [skill.strip() for skill in skills if skill.strip()]

In [10]:
skills = [
    "HTML, CSS, JS",
    "Python/Flask; Django",
    "SQL; Data Analysis, Machine Learning"
]
skills = cleaned_skills

split_skills_result = [split_skills(skill) for skill in skills]
print(split_skills_result)

[['redis'], ['ремонт пк'], ['reindexer'], ['rtps'], ['html'], ['linux'], ['разработка по'], ['bash'], ['linux'], ['css'], ['гост 34'], ['javascript'], ['системный анализ'], ['ci cd'], ['html'], ['gsm'], ['ef core'], ['user story'], ['настройка пк'], ['solid'], ['разработка по'], ['навыки презентации'], ['интерпретация данных'], ['devops'], ['gitlab ci'], ['ci cd'], ['linux'], ['javascript'], ['системный подход'], ['оптимизация бизнес процессов'], ['mysql'], ['ansible'], ['influxdb'], ['сопровождение строительства'], ['test case'], ['настройка dns'], ['npi'], ['функциональное тестирование'], ['unix'], ['docker'], ['kafka'], ['руководство коллективом'], ['управленческая отчетность'], ['кортеос'], ['nginx'], ['javascript'], ['обучение и развитие'], ['системный анализ'], ['elasticsearch'], ['docker'], ['техническая поддержка'], ['администрирование postgresql'], ['ведение деловой переписки'], ['c'], ['mlag'], ['docker'], ['zabbix'], ['agile'], ['развитие продаж'], ['прямые продажи'], ['стро

In [11]:
def cluster_skills(skills):
    """
    Группирует похожие навыки и выбирает представителя для каждого кластера.
    """
    # Преобразование навыков в TF-IDF матрицу
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))  # биграммы и триграммы
    X = vectorizer.fit_transform(skills)
    
    # Кластеризация с помощью HDBSCAN
    clusterer = HDBSCAN(min_cluster_size=2, metric='euclidean')
    labels = clusterer.fit_predict(X)
    
    # Формирование DataFrame для кластеров
    clustered = pd.DataFrame({'original_skill': skills, 'cluster': labels})
    
    # Группировка и выбор представителя
    representatives = clustered.groupby('cluster')['original_skill'].agg(
        lambda group: group.value_counts().idxmax()  # Наиболее частый навык в кластере
    )
    
    # Присваиваем каждому навыку его представителя
    clustered['representative_skill'] = clustered['cluster'].map(representatives)
    
    # Навыки без кластера (шум) оставляем без изменений
    clustered['representative_skill'] = clustered.apply(
        lambda row: row['original_skill'] if row['cluster'] == -1 else row['representative_skill'], axis=1
    )
    
    # Возвращаем словарь {оригинальный навык: представитель}
    return clustered.set_index('original_skill')['representative_skill'].to_dict()


In [12]:
skills = [
    "kubernetes", "Kubernates", "kubernets",
    "Git", " git ", "GIT",
    "HTML", "html", "css", "CSS"
]
skills = split_skills_result

# Кластеризация
skill_map = cluster_skills(skills)

# Замена навыков на представителей
cleaned_skills = [skill_map[skill] for skill in skills]
print(cleaned_skills)


AttributeError: 'list' object has no attribute 'lower'

In [3]:
# Пример данных
df = pd.DataFrame({
    'vacancy_id': [1, 1, 1, 2, 2, 3, 3, 3, 4],
    'professional_role_name': [
        "Developer", "Developer", "Developer",
        "Designer", "Designer",
        "Manager", "Manager", "Manager",
        "Analyst"
    ],
    'key_skill_name': [
        "Kubernetes", "Kubernates", "kubernets",
        "HTML, CSS, JS", "HTML/CSS",
        "Git", " git ", "GIT",
        "Data Analysis"
    ]
})

# --- 1. Очистка текста ---
def clean_symbols(text):
    """Удаление лишних символов, нормализация пробелов и регистра."""
    text = re.sub(r'[^\w\s]', ' ', text)  # Убираем все символы кроме букв, цифр, пробелов
    text = re.sub(r'\s+', ' ', text).strip()  # Нормализуем пробелы
    return text.lower()  # Приводим к нижнему регистру

df['key_skill_name'] = df['key_skill_name'].apply(clean_symbols)

# --- 2. Разделение перечисленных навыков ---
def split_skills(skill_string):
    """Разделение перечисленных навыков."""
    return [skill.strip() for skill in re.split(r'[,\;/]', skill_string) if skill.strip()]

# Разворачиваем перечисленные навыки
df['key_skill_name'] = df['key_skill_name'].apply(split_skills)
df = df.explode('key_skill_name')  # Разделяем строки по отдельным навыкам

# --- 3. Устранение опечаток с помощью HDBSCAN ---
def cluster_skills(skills):
    """Кластеризация похожих навыков и выбор представителя."""
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
    X = vectorizer.fit_transform(skills)
    
    clusterer = HDBSCAN(min_cluster_size=2, metric='euclidean')
    labels = clusterer.fit_predict(X)
    
    # Формируем DataFrame с кластерами
    clustered = pd.DataFrame({'original_skill': skills, 'cluster': labels})
    
    # Выбираем представителя для каждого кластера
    def choose_representative(group):
        return group['original_skill'].value_counts().idxmax()
    
    clustered['representative_skill'] = clustered.groupby('cluster').transform(choose_representative)['original_skill']
    
    # Заменяем -1 (шум) оригинальным значением
    clustered['representative_skill'] = clustered.apply(
        lambda row: row['original_skill'] if row['cluster'] == -1 else row['representative_skill'], axis=1
    )
    
    return clustered.set_index('original_skill')['representative_skill'].to_dict()

# Уникальные навыки
unique_skills = df['key_skill_name'].unique()

# Кластеризация и замена навыков на представителей
skill_map = cluster_skills(unique_skills)
df['key_skill_name'] = df['key_skill_name'].map(skill_map)

# --- 4. Группировка результатов ---
# Группируем по вакансии и роли, объединяя навыки
final_data = df.groupby(['vacancy_id', 'professional_role_name'])['key_skill_name'].apply(list).reset_index()

# Итоговый DataFrame
print(final_data)

KeyError: 'original_skill'

In [8]:
df = df.groupby('vacancy_id').agg({
    'professional_role_name': lambda x: list(set(x)),
    'key_skill_name': lambda x: list(set(x))
}).reset_index()
df

,vacancy_id,professional_role_name,key_skill_name
0,45163827,[Системный администратор],"[Системное мышление, Linux, Nginx, Администрир..."
1,46823861,"[Программист, разработчик]","[Linux, Python, PostgreSQL, Flask, Docker, Redis]"
2,46954274,[Аналитик],"[BPMN, Работа с требованиями, UML, SOAP, Систе..."
3,54961293,[Системный инженер],"[CI/CD, Ansible, Linux, Nginx, Bash, PostgreSQ..."
4,66766808,"[Программист, разработчик]","[JavaScript, TypeScript, Express, REST API, Ne..."
...,...,...,...
5414,111082906,[Системный администратор],"[Желательно: знания сетевой инфраструктуры, оп..."
5415,111099873,[Системный администратор],"[Настройка ПК, Офисная техника, Настройка сете..."
5416,111112257,[Системный администратор],"[IAC, Ansible, MySQL, Terraform, Linux, Debian..."
5417,111124064,[DevOps-инженер],"[Git, CI/CD, MySQL, Linux, DevOps, Grafana, Do..."


In [9]:
vpr_mlb = MultiLabelBinarizer()
X = vpr_mlb.fit_transform(df['professional_role_name'])
X.shape

(5419, 100)

In [10]:
vpr_mlb.classes_

array(['BI-аналитик, аналитик данных', 'DevOps-инженер', 'Event-менеджер',
       'PR-менеджер', 'SMM-менеджер, контент-менеджер',
       'Агент по недвижимости', 'Администратор', 'Аналитик',
       'Андеррайтер', 'Арт-директор, креативный директор', 'Архитектор',
       'Бизнес-аналитик', 'Бизнес-тренер', 'Брокер', 'Бухгалтер',
       'Генеральный директор, исполнительный директор (CEO)',
       'Главный врач, заведующий отделением', 'Главный инженер проекта',
       'Дата-сайентист', 'Делопроизводитель, архивариус',
       'Дизайнер, художник',
       'Директор по информационным технологиям (CIO)',
       'Директор по маркетингу и PR (CMO)', 'Директор по персоналу (HRD)',
       'Директор юридического департамента (CLO)', 'Диспетчер', 'Другое',
       'Инженер ПНР', 'Инженер ПТО, инженер-сметчик',
       'Инженер по качеству', 'Инженер по эксплуатации',
       'Инженер-конструктор, инженер-проектировщик',
       'Инженер-электроник, инженер-электронщик',
       'Инженер-энергетик, ин

In [12]:
vks_mlb = MultiLabelBinarizer()
y = vks_mlb.fit_transform(df['key_skill_name'])
y.shape

(5419, 4766)

In [13]:
vks_mlb.classes_

array(['#devops #sre #kubernetes #k8s #terraform #python #grafana #SiteReliabilityEngineer',
       '- Опыт работы с СУБД mysql, postgresql, MS Access - Опыт работы с IP телефонией. - Знание OC Linux',
       '.NET', ..., '• Управленческие навыки',
       '• Ценообразование и бюджетирование', '\uf02d Tessa'], dtype=object)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(random_state=42)
# clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# print(clf.score(X_test, y_test))

0.0


In [25]:
metric_scorers = {
    'accuracy': 'accuracy'
}

for metric, scorer in metric_scorers.items():
    score = cross_val_score(clf, X, y, cv=4, scoring=scorer).mean()
    print('-', metric, round(score, 3))

- accuracy 0.001
